# Large Survey DataBase

LSDB (Large Survey DataBase) is a framework designed for fast spatial analysis of extremely large astronomical catalogs, enabling efficient querying and crossmatching of over a billion sources. It specifically addresses large-scale data processing challenges, particularly those posed by LSST. Read more about LSDB here.
LSDB is a framework to facilitate and enable spatial analysis for extremely large astronomical databases (i.e. querying and crossmatching O(1B) sources). This package uses dask to parallelize operations across multiple HiPSCat partitioned surveys. 


Please check out:

 - [github.com/astronomy-commons/lsdb](github.com/astronomy-commons/lsdb)
 - [lsdb.io](lsdb.io) (Check out this! There are multiple links to already partitioned catalogs, like GaiaDR3, ZTF and others).

We are applying on S-PLUS a new server side application with lsdb. It allows adding constrains on queries over the network, like selecting specific columns, and filtering results. All the examples below follow this implementation.

We have a walkthrough on how to use LSDB with S-PLUS data available at [splus.cloud/documentation/lsdb_query](https://splus.cloud/documentation/lsdb_query).


### Lets import splusdata and lsdb

In [1]:
import lsdb
import splusdata

Using splusdata we can print the available catalogs, and get the link for one in particular.

In [2]:
# This will print all available links
print(splusdata.get_hipscats())

# This will get the link for a specific pattern
print(splusdata.get_hipscats("dr4/dual"))


{'dr4_vacs': {'qso_z': {'hipscats': ['qso_z'], 'margins': ['qso_z_2arcsec']}, 'photozs': {'hipscats': ['photozs'], 'margins': ['photozs_2arcsec']}, 'calib_flag': {'hipscats': ['calib_flag'], 'margins': ['calib_flag_2arcsec']}, 'sqg': {'hipscats': ['sqg'], 'margins': ['sqg_2arcsec']}}, 'sdr1': {'hipscats': ['shorts'], 'margins': ['shorts_2arcsec']}, 'dr4': {'hipscats': ['single', 'dual', 'psf'], 'margins': ['psf_2arcsec', 'single_2arcsec', 'dual_2arcsec']}}
[['https://splus.cloud/HIPS/catalogs/dr4/dual/', 'https://splus.cloud/HIPS/catalogs/dr4/dual_2arcsec/']]


### Important

Before getting into the code, start a dask Client with the desired specs, this is what does the "parallel magic". This step is very important. It will also print a url to a dashboard that you can use to check the status of your queries.

In [3]:
from dask.distributed import Client

client = Client(n_workers=10, memory_limit="8GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 20,Total memory: 74.51 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59666,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 74.51 GiB
Comm: tcp://127.0.0.1:59696,Total threads: 2
Dashboard: http://127.0.0.1:59699/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:59669,


### Getting data

In [5]:
# This will return a list because there may be more than
# one match for dr4/dual
dr4_dual = splusdata.get_hipscats("dr4/dual")[0]

# dr4_dual will be a tuple, because of the margin link.
dual_link = lsdb.read_hipscat(dr4_dual[0])

In [9]:
print(list(dual_link.columns))

['ID', 'RA', 'DEC', 'A', 'B', 'BACKGROUND', 'BACKGROUND_J0378', 'BACKGROUND_J0395', 'BACKGROUND_J0410', 'BACKGROUND_J0430', 'BACKGROUND_J0515', 'BACKGROUND_J0660', 'BACKGROUND_J0861', 'BACKGROUND_g', 'BACKGROUND_i', 'BACKGROUND_r', 'BACKGROUND_u', 'BACKGROUND_z', 'CLASS_STAR', 'DET_ID_dual', 'EBV_SCH', 'ELLIPTICITY', 'ELONGATION', 'FLUX_RADIUS_20', 'FLUX_RADIUS_50', 'FLUX_RADIUS_70', 'FLUX_RADIUS_90', 'FWHM', 'FWHM_J0378', 'FWHM_J0395', 'FWHM_J0410', 'FWHM_J0430', 'FWHM_J0515', 'FWHM_J0660', 'FWHM_J0861', 'FWHM_g', 'FWHM_i', 'FWHM_n', 'FWHM_n_J0378', 'FWHM_n_J0395', 'FWHM_n_J0410', 'FWHM_n_J0430', 'FWHM_n_J0515', 'FWHM_n_J0660', 'FWHM_n_J0861', 'FWHM_n_g', 'FWHM_n_i', 'FWHM_n_r', 'FWHM_n_u', 'FWHM_n_z', 'FWHM_r', 'FWHM_u', 'FWHM_z', 'Field', 'ID_DEC', 'ID_RA', 'ISOarea', 'J0378_ID_dual', 'J0378_PStotal', 'J0378_aper_3', 'J0378_aper_6', 'J0378_auto', 'J0378_iso', 'J0378_petro', 'J0395_ID_dual', 'J0395_PStotal', 'J0395_aper_3', 'J0395_aper_6', 'J0395_auto', 'J0395_iso', 'J0395_petro', 'J

In [10]:
dr4_links = splusdata.get_hipscats("dr4/dual")[0]

dual_margin = lsdb.read_hipscat(dr4_links[1])
dual = lsdb.read_hipscat(
    dr4_links[0],
    margin_cache=dual_margin,
    columns=["ID", "RA", "DEC", 
             "r_auto", "g_auto", "i_auto", "z_auto", "u_auto",
             "J0378_auto", "J0395_auto", "J0410_auto", "J0430_auto",
             "J0515_auto", "J0660_auto", "J0861_auto"],
    filters=[("r_auto", "<", 18), ("g_auto", ">", 14)],
)

In [16]:
dual.cone_search(
    1, # ra 
    1, # dec
    1*3600 # radius in arcseconds
).compute()

,ID,RA,DEC,r_auto,g_auto,i_auto,z_auto,u_auto,J0378_auto,J0395_auto,J0410_auto,J0430_auto,J0515_auto,J0660_auto,J0861_auto
_hipscat_index,,,,,,,,,,,,,,,
5091979032371658752,DR4_3_STRIPE82-0004_0001327,1.318116,0.062774,16.894825,18.234632,15.964212,15.553219,21.827724,20.961418,20.282782,19.40893,19.271196,18.162392,16.581703,15.673001
5091982231233101824,DR4_3_STRIPE82-0003_0031353,1.111776,0.021377,14.052338,14.577176,13.914072,13.89113,16.121958,15.772111,15.631676,15.046031,14.973824,14.395624,14.034267,13.911386
5091982234492076032,DR4_3_STRIPE82-0004_0000054,1.111787,0.021907,14.08807,14.622419,13.925739,13.852842,16.536917,16.269547,16.295708,15.32154,15.451719,14.789067,14.469679,14.014437
5091982615334879232,DR4_3_STRIPE82-0003_0027945,1.150865,0.028838,17.539032,18.544388,17.092592,16.79575,19.050779,18.839561,17.938129,18.594076,18.210499,17.890163,17.360659,16.743773
5091982615334879233,DR4_3_STRIPE82-0004_0000309,1.150864,0.028842,17.56419,18.787876,17.148464,16.846527,99.0,21.545265,99.0,20.170612,21.277454,18.713846,17.550707,16.940117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477539339124604928,DR4_3_STRIPE82-0004_0031171,1.277447,1.386912,15.365267,15.76194,15.271141,15.28076,17.023418,16.600285,16.603807,16.073318,16.010231,15.639907,15.378314,15.303698
5477539754155180032,DR4_3_STRIPE82-0004_0030375,1.281744,1.412968,14.331291,14.808647,14.19832,14.185552,16.233648,15.874547,15.783333,15.191624,15.138589,14.666297,14.306704,14.208142
5477540480050790400,DR4_3_STRIPE82-0004_0030891,1.180485,1.392128,17.189741,17.494499,17.081303,17.098211,18.586426,18.074364,17.980627,17.624891,17.636324,17.443043,17.187048,17.093132


In [17]:
dual.box_search(
    [-1, 1], # start RA, end RA
    [-1, 1] # start Dec, end Dec
).compute()

,ID,RA,DEC,r_auto,g_auto,i_auto,z_auto,u_auto,J0378_auto,J0395_auto,J0410_auto,J0430_auto,J0515_auto,J0660_auto,J0861_auto
_hipscat_index,,,,,,,,,,,,,,,
4899208249879822336,DR4_3_STRIPE82-0003_0011778,0.964254,-0.984355,16.980633,17.421703,16.841064,16.79199,18.530684,18.187626,18.122524,17.756912,17.710367,17.297626,16.964914,16.806829
4899211249155309568,DR4_3_STRIPE82-0003_0012430,0.97616,-0.96809,15.041121,15.501884,14.899343,14.86096,16.781298,16.417391,16.413391,15.887129,15.844776,15.341377,15.024549,14.879663
4899211261645946880,DR4_3_STRIPE82-0003_0012469,0.97898,-0.962708,16.135672,16.698135,15.943693,15.875214,18.214027,17.91004,17.767872,17.175526,17.187166,16.529642,16.089125,15.906317
4899278485723283456,DR4_3_STRIPE82-0001_0011810,0.276649,-0.991464,15.448023,15.887675,15.306561,15.257038,17.019674,16.641893,16.630896,16.210527,16.190672,15.736286,15.410069,15.257417
4899280014878441472,DR4_3_STRIPE82-0001_0012228,0.402118,-0.977255,17.930283,18.907173,17.592834,17.325251,20.071085,19.379517,19.606922,19.761612,19.422527,18.510283,17.860935,17.489532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477015167298437120,DR4_3_STRIPE82-0002_0044530,359.760099,0.998284,14.808131,15.016464,14.793859,14.838316,16.174797,15.619889,15.43687,15.16936,15.122585,14.894634,14.849438,14.831547
5477019443450085376,DR4_3_STRIPE82-0002_0040702,359.543212,0.988262,16.676786,17.273762,16.509445,16.448231,18.74291,18.595207,18.740599,17.786034,17.674099,17.059698,16.627821,16.473158
5477019576694734848,DR4_3_STRIPE82-0002_0045080,359.581452,0.997842,16.178837,16.673311,16.02849,15.98457,17.960754,17.596251,17.603821,17.060562,17.005465,16.486736,16.147192,15.973917


### Matching two hipscat tables

Assuming you have the the table from the example before, lets instantiate a new hipscat object pointing to another table, in this case Star Galaxy Quasar (sqg) separation table, and make a match with the dual catalog.


In [18]:
dr4_sqg = splusdata.get_hipscats("dr4/sqg")[0]
sqg_margin = lsdb.read_hipscat(dr4_sqg[1])
sqg = lsdb.read_hipscat(
    dr4_sqg[0],
    margin_cache=sqg_margin,
    filters=[("PROB_GAL", ">", 0.8)]
)

/Users/gustavoschwarz/miniconda3/lib/python3.12/site-packages/hipscat/io/file_io/file_io.py:320: UserWarning: point_map.fits file written in RING ordering, due to https://github.com/astronomy-commons/hipscat/issues/271. Converting to NESTED.
  warnings.warn(


In [19]:
dual_sqg = sqg.crossmatch(
    dual, radius_arcsec = 1
)

In [20]:
result_table = dual_sqg.cone_search(
    1, #ra
    1, #dec
    1 * 3600 # radius in arcsecs
).compute()


In [21]:
result_table

,ID_sqg,RA_sqg,DEC_sqg,CLASS_sqg,model_flag_sqg,PROB_QSO_sqg,PROB_STAR_sqg,PROB_GAL_sqg,Norder_sqg,Dir_sqg,...,z_auto_dual,u_auto_dual,J0378_auto_dual,J0395_auto_dual,J0410_auto_dual,J0430_auto_dual,J0515_auto_dual,J0660_auto_dual,J0861_auto_dual,_dist_arcsec
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
5091982615334879232,DR4_3_STRIPE82-0003_0027945,1.150865,0.028838,2.0,0,0.0,0.0,1.0,5,0,...,16.79575,19.050779,18.839561,17.938129,18.594076,18.210499,17.890163,17.360659,16.743773,0.000282
5091982615334879233,DR4_3_STRIPE82-0004_0000309,1.150864,0.028841,2.0,0,0.0,0.01,0.99,5,0,...,16.846527,99.0,21.545265,99.0,20.170612,21.277454,18.713846,17.550707,16.940117,0.000282
5091984044355223552,DR4_3_STRIPE82-0003_0032189,1.204843,0.022827,2.0,0,0.0,0.0,1.0,5,0,...,16.89802,18.517649,18.489357,17.876053,17.87262,18.106815,17.605772,17.325048,16.743799,0.000282
5091985502148493312,DR4_3_STRIPE82-0004_0002515,1.293757,0.09462,2.0,0,0.0,0.0,1.0,5,0,...,16.339067,19.679325,19.227543,18.749567,19.539658,18.906784,17.509869,16.961966,16.473761,0.000282
5091996295271284736,DR4_3_STRIPE82-0003_0041728,0.935803,0.038474,2.0,1,0.0,0.03,0.97,5,0,...,17.094036,99.0,99.0,99.0,21.31982,99.0,19.08293,17.274836,17.269068,0.000282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477516506839384064,DR4_3_STRIPE82-0004_0031617,1.408031,1.376836,2.0,0,0.0,0.0,1.0,1,0,...,17.315907,19.482903,19.000202,18.59066,18.672577,18.517576,17.878046,17.745064,17.27664,0.000282
5477522346325573632,DR4_3_STRIPE82-0004_0030245,1.605574,1.412259,2.0,0,0.0,0.0,1.0,1,0,...,17.270304,20.656267,19.574299,19.082758,19.064711,18.794336,18.104109,17.721167,17.305927,0.000282
5477523454678794240,DR4_3_STRIPE82-0004_0030129,1.515884,1.415197,2.0,0,0.0,0.0,1.0,1,0,...,17.266359,19.315477,19.244976,99.0,20.037548,19.005175,18.162382,17.910954,17.321766,0.000282


### Matching an external table with hipscat

Of course you may also load your own table and match with some S-PLUS table.